In [2]:
import sagemaker
session = sagemaker.Session()
bucket = session.default_bucket()
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
import pandas as pd
import numpy as np

In [4]:
!aws s3 cp s3://sagemaker-sample-files/datasets/tabular/synthetic/churn.txt ./churn.txt

download: s3://sagemaker-sample-files/datasets/tabular/synthetic/churn.txt to ./churn.txt


In [5]:
!head churn.txt

State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
PA,163,806,403-2562,no,yes,300,8.1622040217391,3,7.579173703343681,3.9330349941938625,4,6.508638877091394,4.065759457683862,100,5.1116239145545554,4.9281602056057885,6,5.673203040696216,3,True.
SC,15,836,158-8416,yes,no,0,10.018992664834252,4,4.226288822198435,2.3250045529370977,0,9.97259241534841,7.141039871521733,200,6.436187619334115,3.2217476231887012,6,2.559749162329034,8,False.
MO,131,777,896-6253,no,yes,300,4.708489916519045,3,4.768159882372771,4.53746627560553,3,4.566714501754875,5.363234681368032,100,5.142450865432122,7.139023052596313,2,6.254156588270942,4,False.
WY,75,878,817-5729,yes,yes,700,1.2687337723540173,3,2.567641585633793,2.528747882267883,5,2.3336236298897663,3.773585578112562,450,3.8144134221879837,2.245779327608127,6,1.0806920414898902,6,False

# Read data from CSV

In [6]:
churn_df = pd.read_csv("./churn.txt")
churn_df.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,PA,163,806,403-2562,no,yes,300,8.162204,3,7.579174,...,4,6.508639,4.065759,100,5.111624,4.928160,6,5.673203,3,True.
1,SC,15,836,158-8416,yes,no,0,10.018993,4,4.226289,...,0,9.972592,7.141040,200,6.436188,3.221748,6,2.559749,8,False.
2,MO,131,777,896-6253,no,yes,300,4.708490,3,4.768160,...,3,4.566715,5.363235,100,5.142451,7.139023,2,6.254157,4,False.
3,WY,75,878,817-5729,yes,yes,700,1.268734,3,2.567642,...,5,2.333624,3.773586,450,3.814413,2.245779,6,1.080692,6,False.
4,WY,146,878,450-4942,yes,no,0,2.696177,3,5.908916,...,3,3.670408,3.751673,250,2.796812,6.905545,4,7.134343,6,True.


In [7]:
churn_df["Phone"]

0       403-2562
1       158-8416
2       896-6253
3       817-5729
4       450-4942
          ...   
4995    151-3162
4996    351-5993
4997    370-3127
4998    604-2108
4999    294-2849
Name: Phone, Length: 5000, dtype: object

# Data preprocessing

In [8]:
churn_df = churn_df.drop("Phone",axis=1)

In [9]:
churn_df

,State,Account Length,Area Code,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,PA,163,806,no,yes,300,8.162204,3,7.579174,3.933035,4,6.508639,4.065759,100,5.111624,4.928160,6,5.673203,3,True.
1,SC,15,836,yes,no,0,10.018993,4,4.226289,2.325005,0,9.972592,7.141040,200,6.436188,3.221748,6,2.559749,8,False.
2,MO,131,777,no,yes,300,4.708490,3,4.768160,4.537466,3,4.566715,5.363235,100,5.142451,7.139023,2,6.254157,4,False.
3,WY,75,878,yes,yes,700,1.268734,3,2.567642,2.528748,5,2.333624,3.773586,450,3.814413,2.245779,6,1.080692,6,False.
4,WY,146,878,yes,no,0,2.696177,3,5.908916,6.015337,3,3.670408,3.751673,250,2.796812,6.905545,4,7.134343,6,True.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,NH,4,787,yes,yes,800,10.862632,5,7.250969,6.936164,1,8.026482,4.921314,350,6.748489,4.872570,8,2.122530,9,False.
4996,SD,140,836,no,no,0,1.581127,8,3.758307,7.377591,7,1.328827,0.939932,300,4.522661,6.938571,2,4.600473,4,False.
4997,SC,32,836,no,yes,700,0.163836,5,4.243980,5.841852,3,2.340554,0.939469,450,5.157898,4.388328,7,1.060340,6,False.
4998,MA,142,776,yes,yes,600,2.034454,5,3.014859,4.140554,3,3.470372,6.076043,150,4.362780,7.173376,3,4.871900,7,True.


In [10]:
churn_df["Area Code"]

0       806
1       836
2       777
3       878
4       878
       ... 
4995    787
4996    836
4997    836
4998    776
4999    657
Name: Area Code, Length: 5000, dtype: int64

In [11]:
churn_df["Area Code"] = churn_df["Area Code"].astype(object)

In [12]:
churn_df["Area Code"] 

0       806
1       836
2       777
3       878
4       878
       ... 
4995    787
4996    836
4997    836
4998    776
4999    657
Name: Area Code, Length: 5000, dtype: object

In [13]:
churn_df["Churn?"] = np.where(churn_df["Churn?"] == "False.",0,1)

In [14]:
churn_df = pd.concat(
    [churn_df['Churn?'], churn_df.drop(['Churn?'], axis=1)], axis=1
)
churn_df.head()

,Churn?,State,Account Length,Area Code,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls
0,1,PA,163,806,no,yes,300,8.162204,3,7.579174,3.933035,4,6.508639,4.065759,100,5.111624,4.928160,6,5.673203,3
1,0,SC,15,836,yes,no,0,10.018993,4,4.226289,2.325005,0,9.972592,7.141040,200,6.436188,3.221748,6,2.559749,8
2,0,MO,131,777,no,yes,300,4.708490,3,4.768160,4.537466,3,4.566715,5.363235,100,5.142451,7.139023,2,6.254157,4
3,0,WY,75,878,yes,yes,700,1.268734,3,2.567642,2.528748,5,2.333624,3.773586,450,3.814413,2.245779,6,1.080692,6
4,1,WY,146,878,yes,no,0,2.696177,3,5.908916,6.015337,3,3.670408,3.751673,250,2.796812,6.905545,4,7.134343,6


In [15]:
# use one hot encoding for non numerical fields
churn_df.head()

,Churn?,State,Account Length,Area Code,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls
0,1,PA,163,806,no,yes,300,8.162204,3,7.579174,3.933035,4,6.508639,4.065759,100,5.111624,4.928160,6,5.673203,3
1,0,SC,15,836,yes,no,0,10.018993,4,4.226289,2.325005,0,9.972592,7.141040,200,6.436188,3.221748,6,2.559749,8
2,0,MO,131,777,no,yes,300,4.708490,3,4.768160,4.537466,3,4.566715,5.363235,100,5.142451,7.139023,2,6.254157,4
3,0,WY,75,878,yes,yes,700,1.268734,3,2.567642,2.528748,5,2.333624,3.773586,450,3.814413,2.245779,6,1.080692,6
4,1,WY,146,878,yes,no,0,2.696177,3,5.908916,6.015337,3,3.670408,3.751673,250,2.796812,6.905545,4,7.134343,6


In [16]:
churn_df=pd.get_dummies(churn_df)
churn_df

/opt/conda/lib/python3.10/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


,Churn?,Account Length,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,...,Area Code_858,Area Code_866,Area Code_868,Area Code_876,Area Code_877,Area Code_878,Int'l Plan_no,Int'l Plan_yes,VMail Plan_no,VMail Plan_yes
0,1,163,300,8.162204,3,7.579174,3.933035,4,6.508639,4.065759,...,0,0,0,0,0,0,1,0,0,1
1,0,15,0,10.018993,4,4.226289,2.325005,0,9.972592,7.141040,...,0,0,0,0,0,0,0,1,1,0
2,0,131,300,4.708490,3,4.768160,4.537466,3,4.566715,5.363235,...,0,0,0,0,0,0,1,0,0,1
3,0,75,700,1.268734,3,2.567642,2.528748,5,2.333624,3.773586,...,0,0,0,0,0,1,0,1,0,1
4,1,146,0,2.696177,3,5.908916,6.015337,3,3.670408,3.751673,...,0,0,0,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,4,800,10.862632,5,7.250969,6.936164,1,8.026482,4.921314,...,0,0,0,0,0,0,0,1,0,1
4996,0,140,0,1.581127,8,3.758307,7.377591,7,1.328827,0.939932,...,0,0,0,0,0,0,1,0,1,0
4997,0,32,700,0.163836,5,4.243980,5.841852,3,2.340554,0.939469,...,0,0,0,0,0,0,1,0,0,1
4998,1,142,600,2.034454,5,3.014859,4.140554,3,3.470372,6.076043,...,0,0,0,0,0,0,0,1,0,1


# Data split

In [17]:
churn_df_shuffled = churn_df.sample(frac=1, random_state=42)

In [18]:
#split the dataset into train/validation/test dataset

churn_df_len = len(churn_df_shuffled)

churn_df_train, churn_df_validate,churn_df_test = np.split(churn_df_shuffled,[int(0.6*churn_df_len),int(0.8 * churn_df_len)])




In [19]:
# write datasets to csv
churn_df_train.to_csv('churn_train.csv', header=False, index=False)
churn_df_validate.to_csv('churn_validate.csv', header=False, index=False)

In [20]:
#copy to s3
!aws s3 cp churn_train.csv s3://{bucket}/churn_train.csv
!aws s3 cp churn_validate.csv s3://{bucket}/churn_validate.csv

upload: ./churn_train.csv to s3://sagemaker-us-east-1-524271852738/churn_train.csv
upload: ./churn_validate.csv to s3://sagemaker-us-east-1-524271852738/churn_validate.csv


In [21]:
# create input estimator


from sagemaker.inputs import TrainingInput

s3_input_train = TrainingInput(
    s3_data=f"s3://{bucket}/churn_train.csv", content_type="csv"
)

s3_input_validate = TrainingInput(
    s3_data = f"s3://{bucket}/churn_validate.csv",content_type="csv"
)


In [22]:
# load a container XGBoost algo image

xgb_image = sagemaker.image_uris.retrieve("xgboost",session.boto_region_name, '1.5-1')

In [23]:
# train the model

xgb = sagemaker.estimator.Estimator(
    xgb_image,
    role,
    instance_count=1,
    instance_type = "ml.m5.large",
    output_path=f"s3://{bucket}/output",
    sagemaker_session=session
)

xgb.set_hyperparameters(
    max_depth = 5,
    objective = 'binary:logistic',
    num_round=100
)


xgb.fit(
    {
        'train': s3_input_train,
        'validation': s3_input_validate
    }

)


INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-11-02-16-58-59-224


2023-11-02 16:58:59 Starting - Starting the training job...
2023-11-02 16:59:16 Starting - Preparing the instances for training......
2023-11-02 17:00:25 Downloading - Downloading input data......
2023-11-02 17:01:10 Training - Downloading the training image...
2023-11-02 17:01:51 Training - Training image download completed. Training in progress..[2023-11-02 17:01:56.215 ip-10-0-121-46.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-11-02 17:01:56.239 ip-10-0-121-46.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2023-11-02:17:01:56:INFO] Imported framework sagemaker_xgboost_container.training
[2023-11-02:17:01:56:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2023-11-02:17:01:56:INFO] No GPUs detected (normal if no gpus installed)
[2023-11-02:17:01:56:INFO] Running XGBoost Sagemaker in algorithm mode
[2023-11-02:17:01:56:INFO] Determined 0 GPU(s) available on the inst

In [25]:
from sagemaker.serializers import CSVSerializer

In [27]:
xgb_predictor = xgb.deploy(
    initial_instance_count=1,
    instance_type = 'ml.t2.medium',
    serializer=CSVSerializer()
)

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-11-02-17-03-32-273
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2023-11-02-17-03-32-273
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2023-11-02-17-03-32-273


------------------------------------!

In [28]:
# lets test our model
# convert test dataset into numpy
churn_np_test = churn_df_test.to_numpy()
# the test data set was not used in training
# we can use it to estimate how good is our model at predicting 

In [29]:
actual = churn_np_test[:,0] # this is the first colurm we try to predict 
attributes = churn_np_test[:,1:] # all the other columns are features



In [30]:
res = xgb_predictor.predict(attributes[0]).decode('utf-8')
res

'9.371594933327287e-05\n'

In [32]:
results = xgb_predictor.predict(attributes).decode('utf-8')

In [33]:
predictions_str = results.split("\n")[:-1]
predictions = [float(res_str) for res_str in predictions_str]

In [34]:
predictions[0:10]

[9.371594933327287e-05,
 0.995864987373352,
 0.00017208009376190603,
 0.03127756342291832,
 0.9516103267669678,
 0.15315283834934235,
 9.714097541291267e-05,
 0.996813952922821,
 0.0009423824958503246,
 1.3977289654576452e-06]

In [35]:
# accuracy of our model
from sklearn.metrics import accuracy_score
acc_str = accuracy_score(actual,np.round(predictions))
print(f"The accuracy of our model is {acc_str} %")

The accuracy of our model is 0.948 %


In [36]:
xgb_predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: sagemaker-xgboost-2023-11-02-17-03-32-273
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2023-11-02-17-03-32-273
